In [2]:
# import os
# os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import lsstypes as types
from mockfactory import Catalog
from astropy.table import Table

def read_hdf5_blosc(filename,columns=None,extname='LSS'):
    '''
    read an extension from a hdf5 file that has been blosc compressed
    filename is the full path to the file to read
    columns is the list of columns to read; if None, all will be read
    extname is the extension to read; if None, assumes no separate extensions
    '''
    import h5py
    import hdf5plugin #need to be in the cosmodesi test environment, as of Sep 4th 25
    data = Table()
    with h5py.File(filename) as fn:
        if extname is None:
            if columns is None:
                columns = fn.keys()
            for col in columns:
                data[col] = fn[col][:]

        else:
            if columns is None:
                columns = fn[extname].keys()
            for col in columns:
                data[col] = fn[extname][col][:]

    return data
    
fn = "/global/cfs/cdirs/desi/mocks/cai/LSS/DA2/mocks/holi_v1/altmtl451/loa-v1/mock451/LSScats/QSO_SGC_clustering.dat.h5"
#t = types.read(fn)
#t = Catalog.read(fn)
t = read_hdf5_blosc(fn)

In [5]:
Catalog(data=t)

Catalog(csize=681987, size=681987, columns=['DEC', 'FRAC_TLOBS_TILES', 'NTILE', 'NX', 'PHOTSYS', 'RA', 'TARGETID', 'TILEID', 'WEIGHT', 'WEIGHT_COMP', 'WEIGHT_FKP', 'WEIGHT_IMLIN', 'WEIGHT_SYS', 'WEIGHT_ZFAIL', 'Z'])

In [1]:
from pathlib import Path
from tools import select_region, get_clustering_rdzw, get_clustering_positions_weights
import healpy as hp
import fitsio
import numpy as np
import matplotlib.pyplot as plt

nran=1
region = 'NGC'
weight_type = 'defsault_FKP'

base = Path('/dvs_ro/cfs/cdirs/desi/survey/catalogs/DA2/LSS/loa-v1/LSScats/v2/nonKP/')
data  = get_clustering_positions_weights(base / f'QSO_{region}_clustering.dat.fits',weight_type=weight_type)
randoms  = get_clustering_positions_weights([base / f'QSO_{region}_{iran}_clustering.ran.fits' for iran in range(nran)],weight_type=weight_type)

In [15]:
import jax
from jaxpower import (ParticleField, FKPField, compute_fkp2_normalization, compute_fkp3_normalization, compute_fkp3_shotnoise, 
                      BinMesh3SpectrumPoles, get_mesh_attrs, compute_mesh3_spectrum)

In [3]:
mattrs = get_mesh_attrs(data[0], randoms[0], check=True, cellsize=10, boxsize=10000)

In [4]:
data = ParticleField(data[0],data[1][0], attrs=mattrs, exchange=True, backend='jax')
randoms = ParticleField(randoms[0], randoms[1][0], attrs=mattrs, exchange=True, backend='jax')
fkp = FKPField(data, randoms)

In [22]:
basis='sugiyama-diagonal'
ells=[(0, 0, 0), (2, 0, 2)]
bin = BinMesh3SpectrumPoles(mattrs, edges={'step': 0.01 if 'scoccimarro' in basis else 0.005}, basis=basis, ells=ells, buffer_size=2)  
bin
#norm = compute_fkp3_normalization(fkp, split=42, bin=bin, cellsize=10)

E1222 10:15:49.180298 2298943 pjrt_stream_executor_client.cc:2085] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 7.45GiB. [tf-allocator-allocation-error='']


ValueError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 7.45GiB.

In [18]:
help(compute_fkp3_normalization)

Help on function compute_fkp3_normalization in module jaxpower.mesh3:

compute_fkp3_normalization(*fkps, bin: jaxpower.mesh3.BinMesh3SpectrumPoles = None, cellsize=10.0, split=None)
    Compute the FKP normalization for the bispectrum.

    Parameters
    ----------
    fkps : FKPField or PaticleField
        FKP or particle fields.
    bin : BinMesh3SpectrumPoles, optional
        Binning operator. Only used to return a list of normalization factors for each multipole.
    cellsize : float, optional
        Cell size.
    split : int or None, optional
        Random seed for splitting.
        This is useful to get unbiased estimate of the normalization.
        The input particle fields are split such that the total number of splits is 3.
        For instance, if 3 different fields are provided, no splitting is performed; if 2 fields are provided, one of them is split in 2.
        The each split is painted on a mesh, and the normalization is computed from the product of the 3 meshes

In [13]:
fkp.randoms.positions

Array([[-2781.7405278 ,  1057.80437389,  -531.54038357],
       [-3536.03792715,  1335.53989508,  -675.16879416],
       [-1529.89795193,   578.2317161 ,  -292.39990318],
       ...,
       [ -388.62124087, -1620.40678514,  1047.46470905],
       [ -615.24037198, -2603.82882992,  1689.12455432],
       [ -596.58126891, -2522.93165815,  1639.95757976]], dtype=float64)

In [11]:
randoms.positions

Array([[-2781.7405278 ,  1057.80437389,  -531.54038357],
       [-3536.03792715,  1335.53989508,  -675.16879416],
       [-1529.89795193,   578.2317161 ,  -292.39990318],
       ...,
       [ -388.62124087, -1620.40678514,  1047.46470905],
       [ -615.24037198, -2603.82882992,  1689.12455432],
       [ -596.58126891, -2522.93165815,  1639.95757976]], dtype=float64)

In [11]:
np.array(data[0])

array([[159.24049207, 207.04717952, 207.40245916, ..., 256.64497869,
        256.67821352, 256.70867558],
       [-10.15731177, -10.15902834, -10.17386953, ...,  32.31272065,
         32.34495755,  32.31874305],
       [  3.23773183,   1.45490316,   2.35186215, ...,   0.71793711,
          2.67508203,   1.09902457]])